# 示例解析
## 账本应用
程序在进入时，会创建TradingEnv类的实例env：
```python
env = default.create(
    portfolio=portfolio,
    action_scheme=default.actions.SimpleOrders(),
    reward_scheme=default.rewards.SimpleProfit(),
    feed=feed
)
```
这里我们先来研究action_scheme属性。重点关注evn.action_scheme.action_space属性。由上面的代码可以看出，env.action_scheme是iqt.env.default.actions.SimpleOrders类，action_space是其计算属性，如下所示：
```python
@property
def action_space(self) -> Space:
    if not self._action_space:
        self.actions = product(
            self.criteria,
            self.trade_sizes,
            self.durations,
            [TradeSide.BUY, TradeSide.SELL]
        )
        print('self.actions 1: {0};'.format(self.actions))
        self.actions = list(self.actions)
        print('self.actions 2: {0};'.format(self.actions))
        self.actions = list(product(self.portfolio.exchange_pairs, self.actions))
        print('self.actions 3: {0};'.format(self.actions))
        self.actions = [None] + self.actions
        print('self.actions 4: {0};'.format(self.actions))
        self._action_space = Discrete(len(self.actions))
    return self._action_space
```
运行结果如下所示：
```bash
self.actions 1: <itertools.product object at 0x000001B16C0F53C0>;
self.actions 2: 
[
    (None, 0.1, None, <TradeSide.BUY: 'buy'>), 
    (None, 0.1, None, <TradeSide.SELL: 'sell'>), 
    (None, 0.2, None, <TradeSide.BUY: 'buy'>), 
    ......
    (None, 0.9, None, <TradeSide.SELL: 'sell'>), 
    (None, 1.0, None, <TradeSide.BUY: 'buy'>), 
    (None, 1.0, None, <TradeSide.SELL: 'sell'>)
];

self.actions 3: 
[
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.BUY: 'buy'>)), 
    ......
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.SELL: 'sell'>))
];
    
self.actions 4: 
[
    None, 
    ################################################################################################
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.1, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.2, None, <TradeSide.SELL: 'sell'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.BUY: 'buy'>)), 
    (bitfinex:USD/BTC, (None, 0.3, None, <TradeSide.SELL: 'sell'>)),
    ......
    (bitstamp:USD/LTC, (None, 0.8, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 0.9, None, <TradeSide.SELL: 'sell'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.BUY: 'buy'>)), 
    (bitstamp:USD/LTC, (None, 1.0, None, <TradeSide.SELL: 'sell'>))
];
action_space: Discrete(101);
```
其中action_space为gym.spaces.Descrete(101)，其取值范围为0$\sim$100，对应于step4打印出的某一行，第一项为操作标的，第二项为：
1. cretia：代表市价、限价等订单类型；
2. trade_size：操作比例，如bitstamp: USD/ETH, (None, 0.2, None, <TradeSide.BUY: 'buy'>代表用当前资金的20%买入ETH；
3. duration：None；
4. 买卖方向；

当为1$\sim$100时，代表对交易对的买卖操作。当为0时，代表什么都不做。

接下来对环境进行重置：
```python
def reset(self) -> 'np.array':
    self.episode_id = str(uuid.uuid4())
    self.clock.reset()
    print('components:')
    for c in self.components.values():
        print('    {0};'.format(c))
        if hasattr(c, "reset"):
            c.reset()
    obs = self.observer.observe(self)
    print('init obs: {0};'.format(obs))
    self.clock.increment()
    return obs
```
运行结果如下所示：
```python
components:
    <iqt.env.default.actions.SimpleOrders object at 0x000001C147A5ED00>;
    <iqt.env.default.rewards.SimpleProfit object at 0x000001C147A5EDF0>;
    <iqt.env.default.observers.iqtObserver object at 0x000001C147A5EEB0>;
    <iqt.env.default.stoppers.MaxLossStopper object at 0x000001C147A5EE80>;
    <iqt.env.default.informers.iqtInformer object at 0x000001C147A5E430>;
    <iqt.env.default.renderers.EmptyRenderer object at 0x000001C147A5ED60>;
init obs: [[2569630.2  536484.2]]
```
环境的components包括action_scheme、reward_scheme、observer、stopper（用于确定停止条件，如亏光所有资金）、info（额外信息）、render用于可视化。obs是环境当前状态，在本例中，我们只观察bitstamp交易所的BTC和LTC，其数据文件格式为：

|No.|date|unix|open|high|low|close|volume|
|-|-|-|-|-|-|-|-|
|0|2018/5/15 6:00|1526364000|147.2|148.7|147.01|147.2|1907.28|

我们的obs是bitstamp交易所ETH和LTC两个币种的交易量。

然后进入无限循环，在此无限循环的每一步，首先从action_space中随机取出一个操作，然后调用iqt.env.generic.environment.py::TradingEnv.step方法：
```python
    def step(self, action: Any) -> 'Tuple[np.array, float, bool, dict]':
        """Makes on step through the environment.
        Parameters
        ----------
        action : Any
            An action to perform on the environment.
        Returns
        -------
        `np.array`
            The observation of the environment after the action being
            performed.
        float
            The computed reward for performing the action.
        bool
            Whether or not the episode is complete.
        dict
            The information gathered after completing the step.
        """
        self.action_scheme.perform(self, action)
        obs = self.observer.observe(self)
        reward = self.reward_scheme.reward(self)
        done = self.stopper.stop(self)
        info = self.informer.info(self)
        self.clock.increment()
        return obs, reward, done, info
```

